In [2]:
import sys

!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-h05u6ajw
  Running command git clone -q https://github.com/buguroo/pyknow/ /tmp/pip-req-build-h05u6ajw
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34244 sha256=97070176f96a11890045e9ec4b21f23267e7433d923e53c5088f98ab185083ee
  Stored in directory: /tmp/pip-ephem-wheel-cache-td83m1dk/wheels/d7/8a/1f/40a9edb6f4ce8add0d4029b2c5e274d32d140ac8a7036d5a47
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3166 sha256=4a4784e4cd117fb4b9a8ebb0ef65ca7b79d50ed8c03ebd28268a5b5ff5b7d9d3
  Stored in directory: /root/.cache/pip/wheels/68/17/69/ac196dd181e620bba5fae5488e4fd6366a7316dce13cf88776
Successfully built pyknow frozendict


In [44]:
from pyknow import *

class CrossroadPassage(KnowledgeEngine):
    
    #@DefFacts()
    #def _initial_action(self):
    #    yield Fact(action="greet")

    @Rule(Fact(regulator='Да'))
    def regulator_action(self):
        self.declare(Fact(action = 'Следовать сигналам регулировщика'))

    @Rule(Fact(spec_trans='Да'))
    def spec_trans_action(self):
        self.declare(Fact(action = 'Уступить дорогу транспорту'))

    @Rule(Fact(traffic_lights = 'Да', is_enabled = 'Да'))
    def traf_lights_action(self):
        self.declare(Fact(final_action = 'Следовать сигналам светофора'))
    
    @Rule(AND(NOT(Fact(final_action = W()), 
                       OR(Fact(traffic_light = 'Нет'), Fact(traffic_light = 'Да', is_enabled = 'Нет'), Fact(traffic_light = 'Да', is_enabled = 'Мигает желтый свет')))))
    def check_prior_signs(self):
        self.declare(Fact(action = 'Проверить знаки приоритета'))
    
    @Rule(AND(Fact(action = 'Проверить знаки приоритета'), Fact(prior_signs = 'Да')))
    def prior_signs_action(self):
        self.declare(Fact(final_action = 'Следовать знакам приоритета'))
    
    @Rule(AND(Fact(action = 'Проверить знаки приоритета'), Fact(prior_signs = 'Нет')))
    def check_road_type(self):
        self.declare(Fact(action = 'Проверить покрытие дороги'))
    
    @Rule(AND(Fact(action = 'Проверить покрытие дороги'), Fact(my_road = W()), Fact(another_road = W()), ))
    def check_trans_type(self):
        self.declare(Fact(action = 'Проверить тип ТС'))

    @Rule(AND(Fact(action = 'Проверить покрытие дороги'), Fact(my_road = L('Асфальт') | L('Булыжник') | L('Бетон') | L('Щебёнка'))))
    def final_action1(self):
        self.declare(Fact(final_action = 'Мы на главной дороге'))
    
    @Rule(AND(Fact(action = 'Проверить покрытие дороги'), NOT(Fact(my_road = L('Асфальт') | L('Булыжник') | L('Бетон') | L('Щебёнка')))))
    def final_action2(self):
        self.declare(Fact(final_action = 'Мы на второстепенной дороге'))
    
    @Rule(AND(Fact(action = 'Проверить тип ТС'), Fact(car_type = 'Трамвай')))
    def check_car_type(self):
        self.declare(Fact(action = 'Мы на главной дороге'))

    @Rule(AND(Fact(action = 'Проверить тип ТС'), NOT(Fact(car_type ='Трамвай'))))
    def use_inter_on_right(self):
        self.declare(Fact(action = 'Воспользоваться правилом "помеха справа"'))

    @Rule(AND(Fact(action = 'Воспользоваться правилом "помеха справа"'), Fact(inter_side = 'Справа')))
    def final_action3(self):
        self.declare(Fact(action = 'Мы на второстепенной дороге'))
    
    @Rule(AND(Fact(action = 'Воспользоваться правилом "помеха справа"'), Fact(inter_side = 'Слева')))
    def final_action4(self):
        self.declare(Fact(action = 'Мы на главной дороге'))
    
    def factz(self,l):
        for x in l:
            self.declare(x)

In [45]:
test = CrossroadPassage()
test.reset()

test.factz([
    Fact(regulator='Нет'),
    Fact(spec_trans='Нет'),
    Fact(traffic_light = 'Нет'),
    Fact(prior_signs = 'Нет'),
    Fact(my_road = 'Бетон', another_road = 'вода')
])

test.run()
test.facts

FactList([(0, InitialFact()),
          (1, Fact(regulator='Нет')),
          (2, Fact(spec_trans='Нет')),
          (3, Fact(traffic_light='Нет')),
          (4, Fact(prior_signs='Нет')),
          (5, Fact(my_road='Бетон', another_road='вода')),
          (6, Fact(action='Проверить знаки приоритета')),
          (7, Fact(action='Проверить покрытие дороги')),
          (8, Fact(action='Проверить тип ТС')),
          (9, Fact(action='Воспользоваться правилом "помеха справа"')),
          (10, Fact(final_action='Мы на главной дороге'))])